# Constrained Optimization of the Full Transmon Model

In [1]:
import os
from os.path import join
import sys
import numpy as np
import matplotlib.pylab as plt

In [2]:
%matplotlib inline

In [3]:
%reload_ext autoreload
%autoreload 2

$\newcommand{\Op}[1]{\boldsymbol{\mathsf{\hat{#1}}}}$

## Introduction

We consider the full Transmon Hamiltonian
\begin{equation}
  \Op{H} = \sum_{q=1,2} \left[ \omega_q \Op{b}^{\dagger}_q \Op{b}_q
    + \frac{\alpha_q}{2} \Op{b}^{\dagger}_q\Op{b}^{\dagger}_q\Op{b}_q\Op{b}_q
    + g_q \left(\Op{b}^{\dagger}_q \Op{a} + \Op{b}_q \Op{a}^{\dagger} \right)
    \right]
    + \omega_c \Op{a}^{\dagger}\Op{a}
    + \epsilon(t) \Op{a} + \epsilon^{*}(t) \Op{a}^{\dagger}
\end{equation}
Note that unlike in the optimizations done for KITP in 2013, we do *not* have a static qubit-qubit coupling, outside the effective coupling that is mediated by the cavity

Goals:

1.  Scan the parameter regime systematically for the point that allows the
    easiest implementation of both single- and two-qubit gates. Identify
    known gate mechanisms on map. The scan shows:
    *   Which parameters yield the fastest gates
    *   Which local equivalence classes different qubit parameters lead to
    *   Whether there are "good" regions in addition to the regions at which known gate mechanism have been implemented (determined by resonance conditions)

2.  For select points, determine the quantum speed limit for high fidelity
    gates: What are the fastest possible quantum gates with all-microwave control of transmon qubits 
3.  Include robustness with respect to dissipation, and fluctuations in the
    qubit parameters

Plan:

*   Prerequisite: Method testing
    *   Test optimization with state-dependent and frequency constraints
    *   Test optimization with perfect entangler
    *   Test optimization for single-qubit gate
    *   Examine the RIP method for depopulating the cavity. If generally applicable, incorporate into all guess pulses below
    
*   Scan parameter space.

    We choose $\omega_1$, $\alpha_1 \neq \alpha_2$, $g_1 = g_2$ constant. We vary $\omega_2$ and $\omega_c$ (in the RWA) on a grid. At each grid point (set of paremters), we perform the following steps to evaluate the parameters as a candidate:
    
    *   Set $T=200$ ns (possibly also $T = 500$ ns and $T = 100$ ns
    *   Determine a good guess pulse for the optimization towards a perfect entangler, and for the optimization towards a single-qubit gate
    *   Perform a Krotov or LBFGS optimization, for PE and for single qubit gates (either LI-optimization for identity, or two optimizations $\sigma_x^{(1)}$ and $\sigma_x^{(2)}$.
    
    The critical step is the determination of a good guess pulse. A possiblity is to generate simple guess pulses based on some randomized parameters, and to perform a few simplex steps in these parameters.
    
*  Pick several points in the parameter space that yielded the best success. Determin the QSL by optimizing at increasingly smaller values of $T$.
    

## Environment Setup

In [4]:
%install_ext http://raw.github.com/goerz/version_information/master/version_information.py
%reload_ext version_information
% version_information numpy, matplotlib, pandas, QDYN, QDYNTransmonLib, "cd $PREFIX/src/qdyn && git log",  "cd $PREFIX/src/transmon_oct && git log"

Installed version_information.py. To use it, type:
  %load_ext version_information


Software,Version
Python,2.7.10 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython,3.2.1
OS,Linux 3.13.0 45 generic x86_64 with debian jessie sid
numpy,1.9.2
matplotlib,1.4.3
pandas,0.16.2
QDYN,0.1.2
QDYNTransmonLib,1.0.3
cd $PREFIX/src/qdyn && git log,commit 28d4b4650ec062f00c8ff11179aa276a5cffa9bc
cd $PREFIX/src/transmon_oct && git log,commit 3b6bed6875dbfbbe49d616717bc3f330cdb2709f


In [5]:
PREFIX = join(os.getcwd(), 'venv')
assert os.path.samefile(join(PREFIX, 'bin'), os.environ['PATH'].split(os.pathsep)[0])

## Prerequisites

*   [Does state-dependent constraint work?](Prereq_Nonherm.ipynb) Yes, but slow convergence
*   [Can we ensure a small nq,nc? What sampling rate is sufficient?](Prereq_Cutoff.ipynb) One extra level is sufficient. 100 samples per oscillation is sufficient if we don't claim high precision

## Qubit Paramters

*   $\omega_1/2\pi =$ 6.0 GHz
*   $\omega_2/2\pi$:  varies
*   $\omega_c/2\pi$: varies
*   $\alpha_1/2\pi =$ 290 MHz
*   $\alpha_2/2\pi =$ 310 MHz
*   $g/2\pi =$ 70 MHz
*   $T = $ 200 ns

We take into account 4 qubit levels with realistic decay time [Peterer et al, Phys. Rev. Lett. 114, 010501 (2015)], and 5 cavity levels, also with realistic decay [A. W. Cross and J. M. Gambetta, Phys. Rev. A 91 032325 (2015)]. The last level for each qubit and the cavity is "truncated" by setting the decay to "infinity", as an effective state-dependent constraint.

Decay is modelled by a non-Hermitian Hamiltonian (decay leads to loss of norm)

Possible things to change (discussion with F. Motzoi, 6/23/15):

*   Allow $\omega_2 < \omega_1$
*   Repeat for 100 ns gate duration
*   Going to 11 GHZ with the cavity is a bit unnecessary (numerically too much effort & no new physics)
*   Lowering number of qubit/cavity levels (for numerical benefit): Truncate the eigenstates of the bigger system, as opposed to re-diagonalizing
*   Don't assume JC interaction Hamiltonian: $H_I = g (b^\dagger + b)(a^\dagger + a)$ (cf. Eq 5.14 in thesis)
*   Cavity decay can be brought close to zero, so we might as well use smaller cavity decay
*   Propagating in RWA, checking validity afterwards

## Stage 1: Random Frequency Search & Amplitude Scan

For each parameters pair ($\omega_2, \omega_c$) we run a `./pre_simplex_scan.py` script, taking the following steps:
*   create runfolders, e.g. `w2_6000MHz_wc_11000MHz/stage1/...`
    
        .../field_free -> determine zeta
        .../1_freq_center <- frequency centered between qubits
        .../1_freq_1..10 <- 10 random frequencies
        .../2_freq_resonant <- resonant with both qubits
        .../2_freq_1..10 <- 10 random frequency pairs
        .../5_freq_1..10 <- 10 random frequency/amplitude choices
        [33 runfolders per parameter set]

*   for each runfolder, scan peak amplitude over values $E_0 =$ 10, 50, 100, 150, 200, 250, 300, 350, 400, 450 MHz   
    [total 10 * 33 propagations per parameter set]

*   select best runs, and create runfolders for stage, e.g. `w2_6000MHz_wc_11000MHz/stage1/...`

        .../PE_1freq_center <- best amplitude for perfect entangler
        .../O_1freq_center <- best amplitude for LEC unity
        .../PE_1freq_random <- best random frequency/amplitude for PE
        .../O_1freq_random
        .../PE_2freq_resonant
        .../O_2freq_resonant
        .../PE_2freq_random
        .../O_2freq_random
        .../PE_5freq_random
        .../O_5freq_random
        [10 runfolders per parameter set]
        

In a first attempt, we ran stage 1 in the lab frame, in the folder `./runs_200_LAB`

[Are the obtained pulses still valid in a RWA?](RWA_check.ipynb)

It appears that the concurrence and loss are the same in the lab frame and the RWA frame, except when one of the logical populations drops close to zero. We suspect that in this case, the phases (in which the concurrence depends) is different, and thus we obtain different values in the two frames. Note that the concurrene is not well-defined unless that gate is unitary in the logical subspace.

Thus, from now on, we do everything in the rotating frame:

In [ ]:
! ./run_stage1.py --rwa --duration=100 --parallel=1 --jobs=16 ./runs_100_RWA

[Analysis of the stage-1 results](Stage1Analysis.ipynb) indicate that even by just some random pulses, we can substantially increase and decrease the entanglement from the field-free case. The best results are obtained for the cavity being relatively close to the qubits. i.e., *not* in the dispersive regime!

We prepare for stage-2 by selecting the most promising candidate for each choice of parameters $(\omega_2, \omega_c)$ and for each pulse category:

where the categories `PE_*` maximize entanglement and `SQ_*` minimize entanglement. For each category, we generate a runfolder of the same name, inside a `stage2` subfolder. E.g. `./runs/w2_6100MHz_wc_10100MHz/stage2/PE_2freq_resonant`

In [6]:
! ./select_for_stage2.py ./runs_200_RWA

ERROR: NaN in ./runs_200_RWA/w2_7500MHz_wc_7600MHz/stage1/1freq_2/E900/U.dat
ERROR: increase of norm in ./runs_200_RWA/w2_6310MHz_wc_11100MHz/stage1/5freq_3/E010/U.dat
ERROR: increase of norm in ./runs_200_RWA/w2_5000MHz_wc_11100MHz/stage1/1freq_5/E010/U.dat


In [8]:
! ./select_for_stage2.py ./runs_100_RWA

ERROR: increase of norm in ./runs_100_RWA/w2_7000MHz_wc_6100MHz/stage1/5freq_3/E010/U.dat
ERROR: increase of norm in ./runs_100_RWA/w2_5000MHz_wc_11100MHz/stage1/1freq_7/E010/U.dat
ERROR: increase of norm in ./runs_100_RWA/w2_6500MHz_wc_8600MHz/stage1/5freq_4/E010/U.dat


## Stage 2: Simplex (Pre-)Optimization

For each parameter pair, and each of the 10 runfolders selected for each parameter pair,

*   run simplex search. The optimization functional to be minimized targets either the perfect entanglers or the equivalence class of the identity (local gates):
    \begin{align}
    J^{splx}_{PE} &= \varepsilon_C + \varepsilon_{pop} \\
    J^{splx}_{SQ} &= C + \varepsilon_{pop}
    \end{align}
    where $\varepsilon_C \equiv 1 - C$ is the deviation of the concurrence $C$ from 1 and $\varepsilon_{pop}$ is the loss of population from the logical subspace. 
*   The starting point of the optimization are the pulses selected above; for the different categories, the free parameters are
    * for single-frequency $\epsilon(t) = E_0 B(t) \cos(\omega_L t)$: $E_0$, $\omega_L$ [2 parameters]
    * for 2 frequencies $\epsilon(t) = B(t)(a_1 \cos(\omega_{L,1} t) + a_2 \cos(\omega_{L,2} t + \phi_0)$: $a_1$, $a_2$, $\omega_{L,1}$, $\omega_{L,2}$, $\phi_0$. [5 parameters]
    * for 5 frequencies $\epsilon(t) = B(t)\sum_{n=1}^{5}(a_n \cos(\omega_{L,n} t) + b_n \sin(\omega_{L,n} t))$: $a_{1..5}$, $b_{1..5}$, $\omega_{L,1..5}$ [15 parameters]
    
    All pulses use a Blackman envelope $B(t)$


According to Felix' argument, the same results could have obtained in the RWA. We check whether the gates resulting from the simplex variation yield the same concurrence and loss when propagated in the rotating frame:
[RWA_check.ipynb](RWA_check.ipynb)

In [ ]:
! ./run_stage2.py --rwa --parallel=1 --jobs=30 ./runs_200_RWA

In [ ]:
! ./run_stage2.py --rwa --parallel=1 --jobs=30 ./runs_100_RWA

The optimization used Nelder-Mead-simplex. Powell's method was also tested, but lead to parameters changing into unrealistic regimes.

[Analysis of the stage-2 results](Stage2Analysis.ipynb)

For stage 3, for each point we only consider the best result from the two pulse categories `1freq_random`/`1freq_center`, creating a runfolder `1freq`, and in the same way from `2freq_random`/`2freq_random`, `2freq` is selected. The name for the 5-frequency runfolder is `5freq`

In [ ]:
! ./select_for_stage3.py ./runs_200_RWA

In [ ]:
! ./select_for_stage3.py ./runs_100_RWA

## Stage 3: Gradient-Based Search

We perform 100 OCT iterations for each stage 3 runfolder

In [ ]:
! ./run_stage3.py --rwa --parallel=1 --jobs=200 ./runs_200_RWA

In [ ]:
! ./run_stage3.py --rwa --parallel=1 --jobs=200 ./runs_100_RWA

[Analysis of stage-3 results](Stage3Analysis.ipynb)

The optimization in the RWA surprisingly only yields only gates in a 2-dimensional subset of the Weyl chamber. Possibly the RWA limits the controllability. This puts into question whether RWA and lab frame are really equivalent for the purpose of our analysis. We can perform some checks to get a better idea about controllability in the RWA

## Stage 4: Quantum Speed Limit

For best points resulting from stage 3, lower gate duration until no further gate can be found (to be done)